In [73]:
import requests
import pandas as pd
import json


# API_URL = "https://statsapi.web.nhl.com/api/v1/teams?expand=team.stats&season=19171918"

# Schedule for Dallas Stars 2014-15 season.
API_URL = "https://statsapi.web.nhl.com/api/v1/schedule?season=20142015&teamId=25"

response = requests.get(API_URL)

data = response.json()
# print(json.dumps(data, indent=1))

# print(json.dumps(data['teams'][1]['teamStats'][0]['splits'][0]['stat'], indent=1))

# Get the schedule for the 1917-18 season
# print(data['teams'][1]['teamStats'])

# print(data['teams'][1]['teamStats'][0])
# print(data['teams'][1]['teamStats'][0]['splits'])
# print(data['teams'][1]['teamStats'][0]['splits'][0])
# print(data['teams'][1]['teamStats'][0]['splits'][0]['stat'])
# print(data['teams'][1]['teamStats'][0]['splits'][0]['stat']['wins'])


nhl_teams_colors = {
    'Anaheim Ducks': '#f47a38',
    'Arizona Coyotes': '#8c2633',
    'Atlanta Flames': '#e03a3e',
    'Atlanta Thrashers': '#00214e',
    'Boston Bruins': '#fcb514',
    'Brooklyn Americans': '#002f87',
    'Buffalo Sabres': '#002654',
    'California Golden Seals': '#fdba31',
    'Calgary Flames': '#c8102e',
    'California/Oakland (Golden) Seals': '#fdba31',
    'Carolina Hurricanes': '#cc0000',
    'Chicago Blackhawks': '#cf0a2c',
    'Cleveland Barons': '#ce0f69',
    'Colorado Avalanche': '#6f263d',
    'Colorado Rockies': '#8c2633',
    'Columbus Blue Jackets': '#00285c',
    'Dallas Stars': '#006847',
    'Detroit Cougars': '#ce1126',
    'Detroit Falcons': '#ce1126',
    'Detroit Red Wings': '#ce1126',
    'Edmonton Oilers': '#c8102e',
    'Florida Panthers': '#041e42',
    'Hamilton Tigers': '#ce1126',
    'Hartford Whalers': '#007a33',
    'Kansas City Scouts': '#ce0f69',
    'Los Angeles Kings': '#a2aaad',
    'Minnesota North Stars': '#154734',
    'Minnesota Wild': '#154734',
    'Montréal Canadiens': '#af1e2d',
    'Montréal Maroons': '#af1e2d',
    'Montréal Wanderers': '#af1e2d',
    'Nashville Predators': '#ffb81c',
    'New England Whalers': '#007a33',
    'New Jersey Devils': '#ce1126',
    'New York Americans': '#002f87',
    'New York Islanders': '#f47d30',
    'New York Rangers': '#0038a8',
    'Oakland Seals': '#fdba31',
    'Ottawa Senators (original)': '#e4173e',
    'Ottawa Senators': '#c52032',
    'Philadelphia Flyers': '#f74902',
    'Phoenix Coyotes': '#8c2633',
    'Pittsburgh Pirates': '#000000',
    'Pittsburgh Penguins': '#000000',
    'Québec Nordiques': '#1b4f72',
    'San Diego Gulls': '#fdba31',
    'San Francisco Seals': '#fdba31',
    'San Jose Sharks': '#006d75',
    'Seattle Kraken': '#82b3be',
    'St. Louis Blues': '#002f87',
    'Tampa Bay Lightning': '#002868',
    'Toronto Arenas': '#003e7e',
    'Toronto Maple Leafs': '#003e7e',
    'Vancouver Canucks': '#001f5b',
    'Vegas Golden Knights': '#b4975a',
    'Victoria Cougars': '#ce1126',
    'Washington Capitals': '#041e42',
    'Winnipeg Jets': '#041e42'
}




In [90]:
# Schedule for Dallas Stars 2014-15 season.
# API_URL = "https://statsapi.web.nhl.com/api/v1/schedule?season=20142015&teamId=25"

# Schedule for all teams 2014-15 season.
year = "20222023"
API_URL = "https://statsapi.web.nhl.com/api/v1/schedule?season=" + year

response = requests.get(API_URL)

data = response.json()
# print(json.dumps(data, indent=1))
# print(type(data['dates'][0]))
# print(json.dumps(data['dates'][1], indent=1))

# data['dates'][0] gives you the first date.
# data['dates'][0]['games'] gives you the games for that date.

data['dates'][0]['games'] #[0]['teams']['away']['team']['name']

# Dictionary to hold team names and how many times they occur.
names = {}

# Iterate through all of the dates.
for i in range(len(data['dates'])):
    # Iterate through all of the games for that date.
    for j in range(len(data['dates'][i]['games'])):
        # If the game is a preseason, postseason, or all-star game, skip it.
        if data['dates'][i]['games'][j]['gameType'] == 'PR' or data['dates'][i]['games'][j]['gameType'] == 'P' or data['dates'][i]['games'][j]['gameType'] == 'A':
            continue

        # Get the away team name.
        away_team = data['dates'][i]['games'][j]['teams']['away']['team']['name']

        # Get the home team name.
        home_team = data['dates'][i]['games'][j]['teams']['home']['team']['name']

        # If there is no 'ot', print it and break.
        if 'ot' not in data['dates'][i]['games'][j]['teams']['away']['leagueRecord']:
            print(data['dates'][i]['games'][j]['teams']['away'])
            break

        # If there is no 'ot', print it and break.
        if 'ot' not in data['dates'][i]['games'][j]['teams']['home']['leagueRecord']:
            print(data['dates'][i]['games'][j]['teams']['home'])
            break

        # If the away team name is not in the dictionary, add it.
        if away_team not in names:
            names[away_team] = {'Games': [1]}
            
            # Create new dictionary to keep track of the wins, losses, and ot losses.
            names[away_team] = [sum([data['dates'][i]['games'][j]['teams']['away']['leagueRecord']['wins'] * 2,\
                                              data['dates'][i]['games'][j]['teams']['away']['leagueRecord']['losses'] * 0,\
                                              data['dates'][i]['games'][j]['teams']['away']['leagueRecord']['ot'] * 1])]
        # If the away team name is in the dictionary, increment the count.
        else:
            print(names[away_team]['Games'])
            names[away_team]['Games'].append(names[away_team]['Games'][-1] + 1)
            names[away_team].append(sum([data['dates'][i]['games'][j]['teams']['away']['leagueRecord']['wins'] * 2,\
                                                data['dates'][i]['games'][j]['teams']['away']['leagueRecord']['losses'] * 0,\
                                                data['dates'][i]['games'][j]['teams']['away']['leagueRecord']['ot'] * 1]))

        # If the home team name is not in the dictionary, add it.
        if home_team not in names:
            names[home_team] = {'Games': [1]}
            names[home_team] = [sum([data['dates'][i]['games'][j]['teams']['home']['leagueRecord']['wins'] * 2,\
                                    data['dates'][i]['games'][j]['teams']['home']['leagueRecord']['losses'] * 0,\
                                    data['dates'][i]['games'][j]['teams']['home']['leagueRecord']['ot'] * 1])]
        # If the home team name is in the dictionary, increment the count.
        else:
            names[home_team]['Games'].append(names[home_team]['Games'][-1] + 1)
            names[home_team].append(sum([data['dates'][i]['games'][j]['teams']['home']['leagueRecord']['wins'] * 2,\
                                                  data['dates'][i]['games'][j]['teams']['home']['leagueRecord']['losses'] * 0,\
                                                  data['dates'][i]['games'][j]['teams']['home']['leagueRecord']['ot'] * 1]))

        # Get the away team score.
        away_score = data['dates'][i]['games'][j]['teams']['away']['score']
        # Get the home team score.
        home_score = data['dates'][i]['games'][j]['teams']['home']['score']
test = pd.DataFrame(names, columns=names.keys())

TypeError: list indices must be integers or slices, not str

In [ ]:
print(json.dumps(names, indent=1))

In [82]:
with open('2022-23.json', 'r') as openfile:
 
    # Reading from json file
    test_season = json.load(openfile)

In [83]:
# Now that I have the data, plot it.
import plotly.graph_objs as go
import plotly.offline as pyo
import numpy as np

# Max games played.
# max_games = 0
# for team_name in names.keys():
#     if names[team_name]['Games'] > max_games:
#         max_games = names[team_name]['Games']

# Define your x and y values for each line
x_values = np.arange(1, 83)
data = []

# Define the traces for each line
for team_name in test_season.keys():
    data.append(go.Scatter(x=x_values, y=test_season[team_name], name=team_name, mode='lines', line_color=nhl_teams_colors[team_name]))
# trace_1 = go.Scatter(x=x_values, y=names['Anaheim Ducks']['Points'], name='Anaheim Ducks')
# trace_2 = go.Scatter(x=x_values, y=names['Dallas Stars']['Points'], name='Dallas Stars')
# trace_3 = go.Scatter(x=x_values, y=names['Los Angeles Kings']['Points'], name='Los Angeles Kings')

# Define the layout of the plot
layout = go.Layout(title=f'Team Point Progression ({year[:4]} - {year[4:]} Season)', xaxis=dict(title='Games Played'), yaxis=dict(title='Number of Points'))

# Define the data to be plotted
# data = [trace_1, trace_2, trace_3]

# Create the figure
fig = go.Figure(data=data, layout=layout)

# Display the plot
pyo.plot(fig, filename='multiple-line-graph.html')


San Jose Sharks, [0, 0, 0, 0, 0, 2, 2, 4, 4, 6, 6, 7, 8, 9, 9, 11, 13, 15, 15, 15, 17, 17, 17, 18, 20, 20, 20, 20, 21, 23, 25, 26, 26, 26, 28, 28, 29, 29, 31, 32, 32, 34, 34, 34, 35, 37, 37, 37, 38, 39, 41, 43, 43, 45, 45, 45, 45, 47, 47, 48, 48, 48, 48, 50, 50, 50, 50, 51, 52, 52, 53, 53, 53, 55, 57, 59, 60, 60, 60, 60, 60, 60]
Nashville Predators, [2, 4, 4, 4, 5, 5, 5, 7, 7, 7, 9, 11, 11, 11, 13, 15, 17, 18, 20, 20, 22, 24, 26, 26, 26, 27, 27, 28, 28, 30, 32, 33, 33, 35, 36, 38, 40, 42, 44, 44, 44, 44, 46, 48, 48, 50, 52, 54, 54, 56, 56, 56, 58, 58, 60, 62, 64, 64, 66, 68, 69, 69, 71, 73, 75, 75, 76, 76, 78, 80, 80, 80, 82, 82, 84, 84, 86, 88, 88, 88, 88, 88]
Tampa Bay Lightning, [0, 2, 2, 2, 4, 6, 6, 8, 10, 12, 13, 15, 15, 15, 17, 19, 21, 23, 23, 25, 27, 27, 29, 31, 31, 33, 35, 37, 39, 41, 41, 41, 43, 45, 47, 49, 49, 49, 51, 53, 55, 57, 59, 59, 59, 61, 63, 65, 65, 66, 68, 70, 72, 73, 73, 75, 76, 78, 78, 78, 79, 79, 79, 81, 82, 84, 84, 86, 88, 90, 90, 90, 90, 90, 92, 94, 96, 96, 96, 

'multiple-line-graph.html'

In [89]:
import pandas as pd
# Turn names dictionary into a pandas dataframe.
df = pd.read_json("2022-23.json")
print(df['San Jose Sharks'])

0


                   Team  Games  \
0   Pittsburgh Penguins     82   
1          Dallas Stars     82   
2      New York Rangers     82   
3       Edmonton Oilers     82   
4   Carolina Hurricanes     82   
5         Boston Bruins     82   
6   Toronto Maple Leafs     82   
7    Montréal Canadiens     82   
8       Ottawa Senators     82   
9   Philadelphia Flyers     82   
10    New Jersey Devils     82   
11    Atlanta Thrashers     82   
12   New York Islanders     82   
13  Tampa Bay Lightning     82   
14  Washington Capitals     82   
15     Florida Panthers     82   
16       Buffalo Sabres     82   
17    Detroit Red Wings     82   
18      Phoenix Coyotes     82   
19      St. Louis Blues     82   
20    Los Angeles Kings     82   
21  Nashville Predators     82   
22        Anaheim Ducks     82   
23    Vancouver Canucks     82   
24       Calgary Flames     82   
25      San Jose Sharks     82   
26   Chicago Blackhawks     82   
27   Colorado Avalanche     82   

             

['[2, 4, 6, 6, 8, 8, 8, 8, 10, 12, 12, 12, 12, 14, 14, 16, 16, 16, 16, 16, 17, 19, 19, 21, 23, 25, 27, 27, 29, 31, 31, 33, 33, 35, 35, 35, 35, 37, 39, 41, 43, 43, 45, 46, 48, 48, 50, 52, 54, 54, 56, 58, 60, 62, 62, 64, 66, 66, 68, 68, 70, 71, 71, 73, 75, 75, 75, 76, 78, 80, 82, 82, 84, 86, 87, 89, 91, 92, 92, 92, 92, 92]']